In [3]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
from scipy import sparse
import itertools
%matplotlib inline

!pip install pymorphy2
!pip install textblob
!pip install pyaspeller
!pip install deep_translator
import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

from multiprocessing import Pool
from tqdm import tqdm

import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

from textblob import TextBlob
from pyaspeller import YandexSpeller
from deep_translator import GoogleTranslator

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score

from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegressionCV

import pickle

np.random.seed(42)
random.seed(42)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aleksandrakozevnikova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
train_df = pd.read_csv('train_ml.csv')

In [5]:
test_df = pd.read_csv('new_test_ml.csv')

In [6]:
train_df.head(2)

,bank,feeds,grades,date
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,16.02.2017 16:10
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,13.12.2016 1:05


In [7]:
train_df['date'] = pd.to_datetime(train_df.date)

In [8]:
train_df.date.max()

Timestamp('2021-12-03 23:36:00')

In [9]:
train_df.date.min()

Timestamp('2005-01-07 16:19:00')

In [10]:
train_df.grades.value_counts()# / train_df[train_df.grades >= 1].shape[0]

1.0    27739
5.0    14227
2.0     5634
3.0     2356
4.0     1520
Name: grades, dtype: int64

In [11]:
train_df[(train_df.grades.isna())]

,bank,feeds,grades,date
2,alfabank,Здравствуйте!Ранее уже оставлял отзыв о вашем ...,NaN,2019-06-28 13:54:00
3,vtb,Обращаюсь к Вам с жалобой на незаконное списан...,NaN,2020-07-15 14:54:00
5,pochtabank,Брала кредит на стиральную машину. Все платила...,NaN,2015-09-04 17:19:00
6,otpbank,"Откуда взялся долг по кредитной карте, если я ...",NaN,2021-01-28 13:20:00
9,fk_otkritie,"Уважаемый Бинбанк, если у вас имеются какие-ли...",NaN,2018-08-31 16:48:00
...,...,...,...,...
74981,vtb,13 августа 2018 г. в ДО ЦИК «Каретный Ряд» под...,NaN,2018-05-09 14:09:00
74984,vozrozhdenie,Обман со стороны банка. При снятии наличности ...,NaN,2013-10-02 21:46:00
74985,v-express-bank,Здравствуйте! По действующему ипотечному креди...,NaN,2020-06-23 11:26:00
74988,rsb,"Впервые сталкиваюсь с такой ситуацией, когда б...",NaN,2015-06-18 13:34:00


### Тестовые

In [12]:
test_df.drop(columns=['Unnamed: 0'], inplace=True)

In [13]:
test_df.shape

(17220, 3)

In [14]:
test_df.head()

,bank,feeds,date
0,sberbank,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,01.07.2020 10:53
1,alfabank,Краткое содержание: не рекомендую брать кредит...,20.06.2019 13:19
2,v-express-bank,"Добрый день, уважаемые сотрудники службы контр...",20.02.2016 11:46
3,homecreditbank,"Обращался за получением карты ""Зеленая польза""...",06.05.2019 15:48
4,vtb,20.05.2016 обратилась в отделение банка на про...,23.05.2016 15:41


# Начнем с предобработки

In [18]:
#train_df[:1000].to_excel('train.xlsx', index=False)

In [15]:
stop_words_our = ['а', 'более', 
                 'больше', 'будет',
                 'будто', 'бы',
                 'был', 'была',
                 'были', 'было',
                 'быть', 'вдруг',
                 'во', 'впрочем',
                 'все', 'всегда',
                 'всего', 'всех',
                 'всю', 'г', 'где',
                 'для', 'до', 'другой',
                 'если','есть',
                 'еще', 'ж',
                 'же', 'за',
                 'зачем','из',
                 'или', 'иногда',
                 'к', 'когда',
                 'куда', 'между',
                 'много', 'над',
                 'нибудь', 'о',
                 'об', 'от', 'перед',
                 'по',  'под',
                 'после', 'потом',
                 'почти', 'при',
                 'про', 'разве', 'с',
                 'сейчас', 'со',
                 'тем', 'то', 'только',
                 'том', 'у', 'уж',
                 'уже', 'хоть',
                 'чем', 'чтобы',
                 'чуть', 'я', 'и']

**Исправление орфографических ошибок**

In [16]:
speller = YandexSpeller()
text = test_df.feeds[2]
text

'Добрый день, уважаемые сотрудники службы контроля качества данного " замечательного " банка. Когда же решится сой вопрос с моей кредитной картой? Ранее была выдана мне информация неким сотрудником службы контроля качества Вадимом М., что карта моя выпущена 30.01.2015 года и направлена мне. Далее поступила информация от некого сотрудника также службы контроля качества Козырева Эльдара, который мне сообщил, что карта моя перевыпущена уже 13.01.2016 года и в течении месяца должна быть доставлена мне. Сегодня уже 20. 02.2016 год. Вам самим не смешно без конца просто отписываться мне и кормить завтраками? Когда будет решён мой вопрос? Я воспользуюсь возможность и оставлю претензию на сайте ЦБ Росии о деятельности вашего банка.'

In [17]:
str(TextBlob(text).correct())

'Добрый день, уважаемые сотрудники службы контроля качества данного " замечательного " банка. Когда of решится сой вопрос с моей кредитной картой? Ранее была выдана мне информация неким сотрудником службы контроля качества Вадимом М., что карта моя выпущена 30.01.2015 года и направлена мне. Далее поступила информация of некого сотрудника также службы контроля качества Козырева Эльдара, который мне сообщил, что карта моя перевыпущена уже 13.01.2016 года и в течении месяца должна быть доставлена мне. Сегодня уже 20. 02.2016 год. Вам самим of смешно без конца просто отписываться мне и кормить завтраками? Когда будет решён мой вопрос? Я воспользуюсь возможность и оставлю претензию of сайте of Росии о деятельности вашего банка.'

In [18]:
fixed = speller.spelled(text)
fixed

'Добрый день, уважаемые сотрудники службы контроля качества данного " замечательного " банка. Когда же решится свой вопрос с моей кредитной картой? Ранее была выдана мне информация неким сотрудником службы контроля качества Вадимом М., что карта моя выпущена 30.01.2015 года и направлена мне. Далее поступила информация от некого сотрудника также службы контроля качества Козырева Эльдара, который мне сообщил, что карта моя перевыпущена уже 13.01.2016 года и в течении месяца должна быть доставлена мне. Сегодня уже 20. 02.2016 год. Вам самим не смешно без конца просто отписываться мне и кормить завтраками? Когда будет решён мой вопрос? Я воспользуюсь возможность и оставлю претензию на сайте ЦБ России о деятельности вашего банка.'

Вывод:
Yandex Speller работает лучше

In [19]:
POINTS = {'?' : 'вопрос',
          '!' : 'восклицание'}

In [20]:
def prep_text_new(texts):
    speller = YandexSpeller()
    stop_words = stop_words_our
    regex = re.compile('[^a-z A-z а-я А-Я !?]')
    regex_1 = re.compile('[^a-z а-я]')
    
    preprocess_texts = []
    caps_num = []
    caps_words = []
    for i in tqdm(range(len(texts))):     
        # Поиск CAPS
        caps_counter = 0
        upper_words = []
        for word in texts[i].split():
            if word.isupper():
                caps_counter += 1
                word = regex_1.sub(' ', word.lower())
                if not word in stop_words:
                    upper_words.append(word)
        caps_num.append(caps_counter)
        caps_words.append(' '.join(upper_words))
        
        # Регистр
        text = texts[i].lower()
        text = text.replace('ё', 'е')
        text = text.replace('-', '')
        
        #Удаление веб-адресов:
        text = ' '.join(re.sub("(\w+:\/\/\S+)", "сайт", text).split())
        text = ' '.join(re.sub(r'httpS+', "сайт", text).split())
        
        #Удаление хэштегов / аккаунтов
        text = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", text).split())
        
        # исправление слов с ошибками
        text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
        
        # Иставляем только нужные символы
        text = regex.sub(' ', text)
        
        # исправление орфографических ошибок
        try:
            text = speller.spelled(text)
        except:
            text = str(TextBlob(text).correct())
        
        word_tokens = word_tokenize(text)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        
        # заменим знаки препинания
        prep = ' '.join([POINTS[word] if word in POINTS else word for word in filtered_sentence])
                
        preprocess_texts.append(prep)
    return preprocess_texts, caps_num, caps_words

In [22]:
# запустим предобработку
train_df['prep_feeds'], train_df['cups_num'], train_df['cups_words'] = prep_text_new(train_df.feeds.to_list())

100%|██████████| 75000/75000 [5:08:08<00:00,  4.06it/s]    


In [23]:
train_df.to_csv('prep_train.csv', index = False)

In [24]:
test_df['prep_feeds'], test_df['cups_num'], test_df['cups_words'] = prep_text_new(test_df.feeds.to_list())

100%|██████████| 17220/17220 [1:23:03<00:00,  3.46it/s]


In [25]:
test_df.to_csv('prep_test.csv', index = False)

In [26]:
!pip install pyaspeller
!pip install deep_translator 
!pip install pymorphy2

In [27]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
import itertools
from scipy import sparse
%matplotlib inline

import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

from multiprocessing import Pool
from tqdm import tqdm

import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

from textblob import TextBlob
from pyaspeller import YandexSpeller
from deep_translator import GoogleTranslator

from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate

import pickle


np.random.seed(42)
random.seed(42)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aleksandrakozevnikova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aleksandrakozevnikova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
train_df = pd.read_csv('prep_train.csv')

In [29]:
test_df = pd.read_csv('prep_test.csv')

In [30]:
train_df.head(2)

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,2017-02-16 16:10:00,лет являюсь клиентом этого банка но последний ...,7,отвратительное отношение клиентам ой ужас
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,2016-12-13 01:05:00,ростовнадону ул ленина часов в данном офисе не...,3,г в ао


In [31]:
def stemming_texts(texts):
    st = SnowballStemmer("russian")
    stem_text = []
    for text in tqdm(texts):
        word_tokens = word_tokenize(text)
        stem_text.append(' '.join([st.stem(word) for word in word_tokens]))
    return stem_text

In [32]:
train_df['stem_text'] = stemming_texts(train_df.prep_feeds.tolist())

100%|██████████| 75000/75000 [10:31<00:00, 118.78it/s]


In [33]:
train_df.to_csv('stem_prep_train.csv', index=False)

In [34]:
test_df['stem_text'] = stemming_texts(test_df.prep_feeds.tolist())

100%|██████████| 17220/17220 [02:24<00:00, 118.77it/s]


In [35]:
test_df.to_csv('stem_prep_test.csv', index=False)

In [36]:
train_df = pd.read_csv('stem_prep_train.csv')
train_df.head()

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words,stem_text
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,2017-02-16 16:10:00,лет являюсь клиентом этого банка но последний ...,7,отвратительное отношение клиентам ой ужас,лет явля клиент эт банк но последн виз прост о...
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,2016-12-13 01:05:00,ростовнадону ул ленина часов в данном офисе не...,3,г в ао,ростовнадон ул ленин час в дан офис не оказа н...
2,alfabank,Здравствуйте!Ранее уже оставлял отзыв о вашем ...,NaN,2019-06-28 13:54:00,здравствуйте восклицание ранее оставлял отзыв ...,4,в но а фио,здравств восклицан ран оставля отз ваш банк не...
3,vtb,Обращаюсь к Вам с жалобой на незаконное списан...,NaN,2020-07-15 14:54:00,обращаюсь вам жалобой на незаконное списание д...,30,пао втб уфк пао втб пао втб n фз ...,обраща вам жалоб на незакон списан денежн сред...
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2.0,2020-04-08 06:38:00,имею потребительский кредит взятый в связьбанк...,4,в в псб,им потребительск кред взят в связьбанк перешед...


In [37]:
test_df = pd.read_csv('stem_prep_test.csv')
test_df.head()

,bank,feeds,date,prep_feeds,cups_num,cups_words,stem_text
0,sberbank,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,01.07.2020 10:53,оформляем ипотеку в сбербанке подгружены необх...,0,NaN,оформля ипотек в сбербанк подгруж необходим до...
1,alfabank,Краткое содержание: не рекомендую брать кредит...,20.06.2019 13:19,краткое содержание не рекомендую брать кредит ...,3,в в в,кратк содержан не рекоменд брат кред в эт банк...
2,v-express-bank,"Добрый день, уважаемые сотрудники службы контр...",20.02.2016 11:46,добрый день уважаемые сотрудники службы контро...,3,м цб,добр ден уважа сотрудник служб контрол качеств...
3,homecreditbank,"Обращался за получением карты ""Зеленая польза""...",06.05.2019 15:48,обращался получением карты зеленая польза сотр...,0,NaN,обраща получен карт зелен польз сотрудник сооб...
4,vtb,20.05.2016 обратилась в отделение банка на про...,23.05.2016 15:41,обратилась в отделение банка на проспекте лени...,9,втб втб втб бесплатно колоссальный боль...,обрат в отделен банк на проспект ленин в отдел...


In [38]:
# Выберем для трейна только те коменты, где есть оценка
TRAIN_1 = train_df[train_df.notna().grades]

In [39]:
TRAIN_2 = train_df[train_df.grades.isna()]

In [40]:
vect_words = TfidfVectorizer(max_features=81000, analyzer='word', ngram_range=(1, 1))
vect_chars = TfidfVectorizer(max_features=27600, analyzer='char', ngram_range=(1, 3))

In [41]:
all_words_train = vect_words.fit_transform(TRAIN_1.stem_text)
all_chars_train = vect_chars.fit_transform(TRAIN_1.stem_text)

In [42]:
all_words_train.shape

(51476, 65290)

In [43]:
all_chars_train.shape

(51476, 24988)

In [44]:
# Пропуски в train
all_words_train_2 = vect_words.transform(TRAIN_2.stem_text)
all_chars_train_2 = vect_chars.transform(TRAIN_2.stem_text)

In [45]:
all_words_train_2.shape

(23524, 65290)

In [46]:
all_words_test = vect_words.transform(test_df.stem_text)
all_chars_test = vect_chars.transform(test_df.stem_text)

In [47]:
all_words_test.shape

(17220, 65290)

In [48]:
all_chars_test.shape

(17220, 24988)

In [ ]:
# Соберем матрицу признаков TF-IDF для теста и трейна
train_feats = sparse.hstack([all_words_train, all_chars_train])
train_2_feats = sparse.hstack([all_words_train_2, all_chars_train_2])
test_feats = sparse.hstack([all_words_test, all_chars_test])

In [ ]:
train_feats.shape

In [ ]:
model_LR_CV5 = LogisticRegressionCV(solver='saga', cv=5, scoring='f1_score')
model_LR_CV5.fit(train_feats, TRAIN_1.grades.values)

In [ ]:
scores = cross_validate(model_LR_CV5, train_feats, TRAIN_1.grades.values, cv=5, scoring=('f1_score'), return_train_score=True)
np.mean(scores['test_score'])

In [ ]:
#predicted = model_LR_CV5.predict(train_feats)

In [ ]:
# Сделаем прогноз
test_predicted = model_LR_CV5.predict(test_feats)

In [ ]:
test_predicted = test_predicted.astype(int)

In [ ]:
sol = pd.DataFrame({'inds': test_df.index,
                    'grades': test_predicted})
sol

In [ ]:
sol.to_csv('new_baseline.csv', index=False)

# Переводы

In [6]:
to_translate = 'I want to translate this text'
translated = GoogleTranslator(source='auto', target='ru').translate(to_translate)

In [7]:
translated

'Я хочу перевести этот текст'

# Соберем доп данные путем перевода на английский

In [8]:
import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [9]:
def translate(x, df):
    #result = []
    for it in tqdm.tqdm(x):
        try:
            translated = GoogleTranslator(source='auto', target='en').translate(it)
            tmp = {'comment' : translated}
            df = df.append(tmp, ignore_index=True)
            #result.append(translated)
        except:
            translated = 'miss'
            #result.append(translated)
            tmp = {'comment' : translated}
            df = df.append(tmp, ignore_index=True)
    return df

In [10]:
en_df = pd.DataFrame(columns=['comment'])
en_df

,comment


In [11]:
TRAIN_EN = translate(train_df.feeds.to_list(), en_df)

100%|█████████████████████████████████████████████████████████████████████████| 75000/75000 [14:43:03<00:00,  1.42it/s]


In [12]:
TRAIN_EN.head()

,comment
0,I have been a client of this bank for many yea...
1,"Rostov-on-Don, st. Lenina, 48. Were on Decembe..."
2,Hello! I have already left a review about your...
3,I am writing to you with a complaint about the...
4,I have a consumer loan taken from Svyaz-Bank a...


In [13]:
TRAIN_EN.to_csv('train_en.csv', index=False)

In [14]:
en_df = pd.DataFrame(columns=['comment'])
en_df

,comment


In [15]:
TEST_EN = translate(test_df.feeds.to_list(), en_df)

100%|██████████████████████████████████████████████████████████████████████████| 17220/17220 [2:57:18<00:00,  1.62it/s]


In [16]:
TEST_EN.to_csv('test_en.csv', index=False)

In [2]:
TRAIN_EN = pd.read_csv('train_en.csv')

In [4]:
TRAIN_EN.head()

,comment
0,I have been a client of this bank for many yea...
1,"Rostov-on-Don, st. Lenina, 48. Were on Decembe..."
2,Hello! I have already left a review about your...
3,I am writing to you with a complaint about the...
4,I have a consumer loan taken from Svyaz-Bank a...


In [10]:
# теперь назад
def translate_ru(x):
    result = []
    for it in tqdm(x):
        try:
            translated = GoogleTranslator(source='auto', target='ru').translate(it)
            #tmp = {'comment' : translated}
            #df = df.append(tmp, ignore_index=True)
            result.append(translated)
        except:
            translated = 'miss'
            result.append(translated)
            #tmp = {'comment' : translated}
            #df = df.append(tmp, ignore_index=True)
    return result

In [11]:
#en_df = pd.DataFrame(columns=['comment'])
#en_df

In [ ]:
TRAIN_EN['ru'] = translate_ru(TRAIN_EN.comment.to_list())

 70%|█████████████████████████████████████████████████▋                     | 52421/75000 [19:26:00<7:17:35,  1.16s/it]

In [ ]:
TRAIN_EN.head()

In [ ]:
TRAIN_EN.to_csv('train_en_ru.csv', index=False)

In [ ]:
TEST_EN = pd.read_csv('test_en.csv')

In [ ]:
TEST_EN['ru'] = translate_ru(TEST_EN.comment.to_list())

In [ ]:
TEST_EN.to_csv('test_en_ru.csv', index=False)

# Обучим модель № 5

# Продолжим работу из colab

In [80]:
TRAIN_W_ADD = pd.read_csv('TRAIN_W_ADD_RATING.csv')

test_df = pd.read_csv('test_df_RATING.csv')

# TF-IDF

In [3]:
vect_words = TfidfVectorizer(max_features=50000, analyzer='word', ngram_range=(1, 3))
vect_chars = TfidfVectorizer(max_features=25000, analyzer='char', ngram_range=(1, 3))

In [4]:
all_words_train = vect_words.fit_transform(TRAIN_W_ADD.stem_text)
all_chars_train = vect_chars.fit_transform(TRAIN_W_ADD.stem_text)

In [5]:
all_words_test = vect_words.transform(test_df.stem_text)
all_chars_test = vect_chars.transform(test_df.stem_text)

In [6]:
# Соберем матрицу признаков TF-IDF для теста и трейна
train_feats = sparse.hstack([all_words_train, all_chars_train])
test_feats = sparse.hstack([all_words_test, all_chars_test])

In [7]:
train_feats.shape

(70827, 75000)

In [8]:
with open('model_tfidf_BigData_3.pickle', 'rb') as f:
    model_3_LR_CV3 = pickle.load(f)

C:\Users\kulak\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LogisticRegressionCV from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
# Посмотрим на признаки
coef = model_3_LR_CV3.coef_.mean(axis=0)

In [10]:
new_train_features = train_feats.tocsr()[:, (np.abs(coef) > 1e-15)]

In [11]:
new_test_features = test_feats.tocsr()[:, (np.abs(coef) > 1e-15)]

In [12]:
new_train_features.shape

(70827, 8728)

# One hot encoding

In [13]:
ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [0, 1, 2])],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough')

In [14]:
ohe_train = ct.fit_transform(TRAIN_W_ADD.loc[:, ['bank', 'y', 'm', 'cups_num', 'rating', 'len']])

In [15]:
ohe_train.shape

(70827, 101)

In [16]:
test_df.head()

,bank,feeds,date,prep_feeds,cups_num,cups_words,stem_text,y,m,rating,len
0,sberbank,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,2020-01-07 10:53:00,оформляем ипотеку в сбербанке подгружены необх...,0,NaN,оформля ипотек в сбербанк подгруж необходим до...,2020,1,1.767017,331
1,alfabank,Краткое содержание: не рекомендую брать кредит...,2019-06-20 13:19:00,краткое содержание не рекомендую брать кредит ...,3,в в в,кратк содержан не рекоменд брат кред в эт банк...,2019,6,1.942431,1048
2,v-express-bank,"Добрый день, уважаемые сотрудники службы контр...",2016-02-20 11:46:00,добрый день уважаемые сотрудники службы контро...,3,м цб,добр ден уважа сотрудник служб контрол качеств...,2016,2,1.120879,731
3,homecreditbank,"Обращался за получением карты ""Зеленая польза""...",2019-06-05 15:48:00,обращался получением карты зеленая польза сотр...,0,NaN,обраща получен карт зелен польз сотрудник сооб...,2019,6,2.659696,415
4,vtb,20.05.2016 обратилась в отделение банка на про...,2016-05-23 15:41:00,обратилась в отделение банка на проспекте лени...,9,втб втб втб бесплатно колоссальный боль...,обрат в отделен банк на проспект ленин в отдел...,2016,5,1.885882,2510


In [17]:
ohe_test = ct.transform(test_df.loc[:, ['bank', 'y', 'm', 'cups_num', 'rating', 'len']])

# Склеим данные

In [18]:
# Склеим данные
train_sparse = sparse.hstack([new_train_features, ohe_train])

In [19]:
test_sparse = sparse.hstack([new_test_features, ohe_test])

In [20]:
train_sparse.shape

(70827, 8829)

In [21]:
test_sparse.shape

(17220, 8829)

# Попробуем что-нибудь обучить

In [22]:
TRAIN_W_ADD.head()

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words,stem_text,y,m,rating,len
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1,2017-02-16 16:10:00,лет являюсь клиентом этого банка но последний ...,7,отвратительное отношение клиентам ой ужас,лет явля клиент эт банк но последн виз прост о...,2017,2,1.421053,1286
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2,2016-12-13 01:05:00,ростовнадону ул ленина часов в данном офисе не...,3,г в ао,ростовнадон ул ленин час в дан офис не оказа н...,2016,12,1.652968,1000
2,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2,2020-04-08 06:38:00,имею потребительский кредит взятый в связьбанк...,4,в в псб,им потребительск кред взят в связьбанк перешед...,2020,4,1.945946,1614
3,homecreditbank,Хочу выразить глубокую благодарность всем сотр...,5,2015-04-19 20:11:00,хочу выразить глубокую благодарность всем сотр...,0,NaN,хоч выраз глубок благодарн всем сотрудник высо...,2015,4,1.473498,438
4,vtb,"6 марта 2019г. В 10:33 при проходе в метро ""Ку...",2,2019-03-28 18:36:00,марта в проходе в метро курская социальной кар...,3,в втб,март в проход в метр курск социальн карт беспл...,2019,3,2.236177,1175


In [28]:
clf1 = LogisticRegression(multi_class='multinomial', random_state=1)
clf2 = LinearSVC()
clf3 = GaussianNB()

eclf1 = VotingClassifier(estimators=[('lr', clf1), ('svc', clf2), ('gnb', clf3)], voting='soft')

In [36]:
scores = cross_validate(eclf1, train_sparse, TRAIN_W_ADD.grades.values, cv=3,
                         scoring=('f1_micro'),
                         return_train_score=True)

C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preproces

ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\kulak\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\kulak\anaconda3\lib\site-packages\sklearn\ensemble\_voting.py", line 351, in fit
    return super().fit(X, transformed_y, sample_weight)
  File "C:\Users\kulak\anaconda3\lib\site-packages\sklearn\ensemble\_voting.py", line 83, in fit
    self.estimators_ = Parallel(n_jobs=self.n_jobs)(
  File "C:\Users\kulak\anaconda3\lib\site-packages\joblib\parallel.py", line 1046, in __call__
    while self.dispatch_one_batch(iterator):
  File "C:\Users\kulak\anaconda3\li

In [ ]:
np.mean(scores['test_score'])

In [25]:
model_5_LR_CV7 = LogisticRegressionCV(solver='sag', cv=7, scoring='f1_micro')

In [26]:
model_5_LR_CV7.fit(new_train_features, TRAIN_W_ADD.grades.values)

C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter 

LogisticRegressionCV(cv=7, scoring='f1_micro', solver='sag')

# Валидация

In [32]:
from sklearn.metrics import f1_score

In [33]:
predicted = model_5_LR_CV7.predict(new_train_features)

In [34]:
f1_score(predicted, TRAIN_W_ADD.grades.values, average='micro')

0.8644867070467477

In [30]:
# Сохраним модельку
with open('model_5_coef_OHE.pickle', 'wb') as f:
    pickle.dump(model_5_LR_CV7, f)

In [31]:
# Сделаем прогноз
#test_predicted = model_5_LR_CV3.predict(test_sparse)

In [ ]:
#test_predicted = test_predicted.astype(int)

In [ ]:
#sol = pd.DataFrame({'inds': test_df.index,
#                    'grades': test_predicted})

In [ ]:
#sol

In [ ]:
#sol.to_csv('solution_3.csv', index=False)

# Поработаем с доразметкой

In [36]:
path = 'Неуверенные прогнозы на трейне/UNSURE_tr_2_wt_first.xlsx'

In [37]:
unsure = pd.read_excel(path)

In [38]:
unsure.head()

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words,stem_text,proba
0,homecreditbank,Несколько лет с удовольствием пользовалась усл...,3,2015-02-10 19:29:00,несколько лет удовольствием пользовалась услуг...,0,NaN,нескольк лет удовольств пользова услуг эт банк...,0.452553
1,unicreditbank,"Без оценки по причине, что так и допожно быть ...",5,2014-03-25 00:35:00,без оценки причине что так допожно в нормальны...,1,в,без оценк причин что так допожн в нормальн бан...,0.447934
2,rsb,Был клиентом огромного кол-ва финансовых учреж...,5,2015-02-09 11:32:00,клиентом огромного колва финансовых учреждений...,2,в в,клиент огромн колв финансов учрежден бизнес по...,0.722269
3,alfabank,Вопрос:Добрый день! Я страдаю от тяжелой депре...,3,2020-09-27 21:27:00,вопрос добрый день восклицание страдаю тяжелой...,7,а,вопрос добр ден восклицан страда тяжел депресс...,0.421834
4,avangard,Партнёрствую с Авангардом уже более 10 лет и к...,5,2018-04-12 12:25:00,партнерствую авангардом лет как физ как юрлицо...,2,NaN,партнерств авангард лет как физ как юрлиц стал...,0.447373


In [41]:
unsure.iloc[:414, :]

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words,stem_text,proba
0,homecreditbank,Несколько лет с удовольствием пользовалась усл...,3,2015-02-10 19:29:00,несколько лет удовольствием пользовалась услуг...,0,NaN,нескольк лет удовольств пользова услуг эт банк...,0.452553
1,unicreditbank,"Без оценки по причине, что так и допожно быть ...",5,2014-03-25 00:35:00,без оценки причине что так допожно в нормальны...,1,в,без оценк причин что так допожн в нормальн бан...,0.447934
2,rsb,Был клиентом огромного кол-ва финансовых учреж...,5,2015-02-09 11:32:00,клиентом огромного колва финансовых учреждений...,2,в в,клиент огромн колв финансов учрежден бизнес по...,0.722269
3,alfabank,Вопрос:Добрый день! Я страдаю от тяжелой депре...,3,2020-09-27 21:27:00,вопрос добрый день восклицание страдаю тяжелой...,7,а,вопрос добр ден восклицан страда тяжел депресс...,0.421834
4,avangard,Партнёрствую с Авангардом уже более 10 лет и к...,5,2018-04-12 12:25:00,партнерствую авангардом лет как физ как юрлицо...,2,NaN,партнерств авангард лет как физ как юрлиц стал...,0.447373
...,...,...,...,...,...,...,...,...,...
409,modulbank,"Здравствуйте,У меня строительная организация ...",2,2019-07-27 14:50:00,здравствуйте меня строительная организация зан...,0,NaN,здравств мен строительн организац занима тендо...,0.439591
410,trust,Являюсь вкладчиком с весны. Тысячу раз уже пож...,1,2013-10-14 19:08:00,являюсь вкладчиком весны тысячу раз пожалел эт...,3,NaN,явля вкладчик весн тысяч раз пожалел эт питер ...,0.389042
411,v-express-bank,В субботу 08.04 я пришла в отделение банка на ...,1,2017-10-04 14:59:00,в субботу пришла в отделение банка на комсомол...,2,в,в суббот пришл в отделен банк на комсомольск п...,0.340138
412,tcs,"Здравствуйте, оставил заявку на сайте на получ...",2,2012-09-26 00:30:00,здравствуйте оставил заявку на сайте на получе...,0,NaN,здравств остав заявк на сайт на получен кредит...,0.282219


In [39]:
all_words_unsure = vect_words.transform(unsure.stem_text)
all_chars_unsure = vect_chars.transform(unsure.stem_text)

In [42]:
# Соберем матрицу признаков TF-IDF для теста и трейна
unsure_feats = sparse.hstack([all_words_unsure, all_chars_unsure])

In [43]:
unsure_feats.shape

(1339, 75000)

In [55]:
valid_unsure = unsure_feats.tocsr()[:414, :]
test_insure = unsure_feats.tocsr()[414:, :]

In [51]:
y_valid = unsure.loc[:413, ['grades']].values

In [52]:
y_valid.shape

(414, 1)

In [46]:
valid.shape

(414, 75000)

In [47]:
test.shape

(925, 75000)

## Оставим только нужные признаки

In [56]:
new_valid_unsure_features = valid_unsure.tocsr()[:, (np.abs(coef) > 1e-15)]
new_test_unsure_features = test_insure.tocsr()[:, (np.abs(coef) > 1e-15)]

In [57]:
new_valid_unsure_features.shape

(414, 8728)

# Проверим нашу модель

In [58]:
pred_unsure_valid = model_5_LR_CV7.predict(new_valid_unsure_features)

In [59]:
f1_score(pred_unsure_valid, y_valid, average='micro')

0.35990338164251207

# Вот и реальное качество модели

In [63]:
train_unsure_df = unsure.iloc[:414,:]

In [65]:
test_unsure_df = unsure.iloc[414:,:]

In [67]:
train_unsure_df.grades.value_counts()

5    111
3    107
4     76
2     65
1     55
Name: grades, dtype: int64

In [78]:
# Обучим модель на этих данных
model_6_unsure_LR_CV3 = LogisticRegressionCV(solver='saga', cv=5, scoring='f1_micro')

In [79]:
model_6_unsure_LR_CV3.fit(valid_unsure, y_valid)

C:\Users\kulak\anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anacond

C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter 

LogisticRegressionCV(cv=5, scoring='f1_micro', solver='saga')

In [81]:
pred_unsure_valid = model_6_unsure_LR_CV3.predict(valid_unsure)

f1_score(pred_unsure_valid, y_valid, average='micro')

0.6521739130434783

In [82]:
pred_test_unsure = model_6_unsure_LR_CV3.predict(test_insure)

In [83]:
test_unsure_df['new_grades'] = pred_test_unsure

<ipython-input-83-cf8d11e8c812>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_unsure_df['new_grades'] = pred_test_unsure


In [84]:
test_unsure_df.to_excel('Неуверенные прогнозы на трейне/test_unsure_pred.xlsx', index=False)

# Попробуем разбить данные подругому

In [2]:
test_df = pd.read_csv('prep_test.csv')
train_df = pd.read_csv('prep_train.csv')

In [3]:
train_en = pd.read_csv('train_en.csv')

In [4]:
test_en = pd.read_csv('test_en.csv')

In [5]:
train_en.head()

,comment
0,I have been a client of this bank for many yea...
1,"Rostov-on-Don, st. Lenina, 48. Were on Decembe..."
2,Hello! I have already left a review about your...
3,I am writing to you with a complaint about the...
4,I have a consumer loan taken from Svyaz-Bank a...


# Переводы

In [6]:
train_df['en'] = train_en.comment.values

In [7]:
test_df['en'] = test_en.comment.values

In [8]:
train_df.head()

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words,en
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,2017-02-16 16:10:00,лет являюсь клиентом этого банка но последний ...,7,отвратительное отношение клиентам ой ужас,I have been a client of this bank for many yea...
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,2016-12-13 01:05:00,ростовнадону ул ленина часов в данном офисе не...,3,г в ао,"Rostov-on-Don, st. Lenina, 48. Were on Decembe..."
2,alfabank,Здравствуйте!Ранее уже оставлял отзыв о вашем ...,NaN,2019-06-28 13:54:00,здравствуйте восклицание ранее оставлял отзыв ...,4,в но а фио,Hello! I have already left a review about your...
3,vtb,Обращаюсь к Вам с жалобой на незаконное списан...,NaN,2020-07-15 14:54:00,обращаюсь вам жалобой на незаконное списание д...,30,пао втб уфк пао втб пао втб n фз ...,I am writing to you with a complaint about the...
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2.0,2020-04-08 06:38:00,имею потребительский кредит взятый в связьбанк...,4,в в псб,I have a consumer loan taken from Svyaz-Bank a...


In [9]:
test_df.head()

,bank,feeds,date,prep_feeds,cups_num,cups_words,en
0,sberbank,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,01.07.2020 10:53,оформляем ипотеку в сбербанке подгружены необх...,0,NaN,We arrange a mortgage in Sberbank. On 06/22/20...
1,alfabank,Краткое содержание: не рекомендую брать кредит...,20.06.2019 13:19,краткое содержание не рекомендую брать кредит ...,3,в в в,Short content: I do not recommend taking a loa...
2,v-express-bank,"Добрый день, уважаемые сотрудники службы контр...",20.02.2016 11:46,добрый день уважаемые сотрудники службы контро...,3,м цб,"Good afternoon, dear employees of the quality ..."
3,homecreditbank,"Обращался за получением карты ""Зеленая польза""...",06.05.2019 15:48,обращался получением карты зеленая польза сотр...,0,NaN,"I applied for a ""Green Benefit"" card, the empl..."
4,vtb,20.05.2016 обратилась в отделение банка на про...,23.05.2016 15:41,обратилась в отделение банка на проспекте лени...,9,втб втб втб бесплатно колоссальный боль...,"On May 20, 2016, she applied to the bank branc..."


# Предобработка


In [10]:
train_notna = train_df[train_df.grades.notna()]

In [11]:
train_notna.grades.value_counts()

1.0    27739
5.0    14227
2.0     5634
3.0     2356
4.0     1520
Name: grades, dtype: int64

# Предобработаем и обучим модель на английском


In [12]:
train_notna.en[0]

"I have been a client of this bank for many years, but the last visit just left a negative in my soul! As a bank client, I was offered a credit card, after thinking about the offer, I decided to issue it. 09.02 I went to the office at Yekaterinburg, b. Ak. Bardina, 17, filled out an application, they said to wait 15 minutes, after approval came to draw up. The specialist who worked with me said that I needed to pay 900 rubles for servicing the card, I asked her why they didn’t warn me in advance, to which she smiled at me with a smirk. HORRIBLE CUSTOMER TREATMENT! It gets even worse ((After specifying how long the approval for the application is valid (10 days), we agreed that I would come on another day! On 13.02 I come to the bank, the same employee cannot print the contract, smiling, she says to me: OH, what- then the loan doesn’t want to be approved for you?! (Let’s fill out a new application for you on general terms, in general, there is a refusal ((Chuckling, she told me to wait 

In [13]:
m = MorphAnalyzer()
regex = re.compile("[A-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('english') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [61]:
lemmas = []
for i, text in tqdm(enumerate(train_df['en'].tolist())):
    lemmas.append(clean_text(text))

75000it [02:22, 525.22it/s]


In [62]:
train_df['lemmas'] = lemmas

In [28]:
lemmas = []
for i, text in tqdm(enumerate(test_df['en'].tolist())):
    lemmas.append(clean_text(text))

17220it [00:32, 527.67it/s]


In [29]:
test_df['lemmas'] = lemmas

In [65]:
train_df.to_csv('train_prep_en.csv', index=False)
test_df.to_csv('test_prep_en.csv', index=False)

In [ ]:
#with Pool(4) as p:
#    lemmas = list(tqdm(p.imap(clean_text, train_notna['en']), total=len(train_notna)))
    
#train_notna['lemmas'] = lemmas
#train_notna.sample(5)

  0%|                                                                                        | 0/51476 [00:00<?, ?it/s]

In [68]:
miss = train_df[train_df.grades.isna()]

# TF-IDF

In [38]:
vect_words = TfidfVectorizer(max_features=250000, analyzer='word', ngram_range=(1, 3))
vect_chars = TfidfVectorizer(max_features=12000, analyzer='char', ngram_range=(1, 3))

In [39]:
all_words_train = vect_words.fit_transform(train_notna.lemmas)
all_chars_train = vect_chars.fit_transform(train_notna.lemmas)

In [40]:
all_words_train.shape

(51476, 250000)

In [34]:
all_chars_train.shape

(51476, 11291)

In [41]:
all_words_test = vect_words.transform(test_df.lemmas)
all_chars_test = vect_chars.transform(test_df.lemmas)

In [69]:
all_words_miss = vect_words.transform(miss.lemmas)
all_chars_miss = vect_chars.transform(miss.lemmas)

In [70]:
miss_feats = sparse.hstack([all_words_miss, all_chars_miss])

In [42]:
# Соберем матрицу признаков TF-IDF для теста и трейна
train_feats = sparse.hstack([all_words_train, all_chars_train])
test_feats = sparse.hstack([all_words_test, all_chars_test])

# Обучим модель

In [48]:
# Обучим модель на этих данных
model_7_en_LR_CV5 = LogisticRegressionCV(solver='saga', cv=5)

In [49]:
model_7_en_LR_CV5.fit(train_feats, train_notna.grades.values)

C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kulak\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter 

LogisticRegressionCV(cv=5, solver='saga')

In [50]:
pred_train_en = model_7_en_LR_CV5.predict(train_feats)

f1_score(pred_train_en, train_notna.grades.values, average='micro')

0.9168544564457223

# Сохраним предикт

In [52]:
# Сохраним модельку
# tf-idf word250k+char12k
with open('model_7_en.pickle', 'wb') as f:
    pickle.dump(model_7_en_LR_CV5, f)

In [53]:
# Сделаем прогноз
pred_test = model_7_en_LR_CV5.predict(test_feats)

In [54]:
pred_test = pred_test.astype(int)

In [56]:
sol = pd.DataFrame({'inds': test_df.index,
                    'grades': pred_test})

In [58]:
sol.head()

,inds,grades
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [59]:
sol.to_csv('solution_3.csv', index=False)

In [71]:
# Сделаем прогноз на пропусках
pred_miss = model_7_en_LR_CV5.predict(miss_feats)

In [75]:
pred_miss_prob = model_7_en_LR_CV5.predict_proba(miss_feats)

In [73]:
miss.grades = pred_miss

<ipython-input-73-2cfa22ab6d4a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miss.grades = pred_miss


In [98]:
probs = pred_miss_prob.max(axis=1)

In [99]:
miss['prob'] = probs

<ipython-input-99-f844556b3274>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miss['prob'] = probs


In [100]:
miss.head()

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words,en,lemmas,prob
2,alfabank,Здравствуйте!Ранее уже оставлял отзыв о вашем ...,1.0,2019-06-28 13:54:00,здравствуйте восклицание ранее оставлял отзыв ...,4,в но а фио,Hello! I have already left a review about your...,hello already left review bank incompetence em...,0.841549
3,vtb,Обращаюсь к Вам с жалобой на незаконное списан...,1.0,2020-07-15 14:54:00,обращаюсь вам жалобой на незаконное списание д...,30,пао втб уфк пао втб пао втб n фз ...,I am writing to you with a complaint about the...,writing complaint illegal debiting funds child...,0.976208
5,pochtabank,Брала кредит на стиральную машину. Все платила...,1.0,2015-09-04 17:19:00,брала кредит на стиральную машину платила в ср...,2,NaN,I took out a loan for a washing machine. Every...,took loan washing machine everything paid time...,0.794737
6,otpbank,"Откуда взялся долг по кредитной карте, если я ...",1.0,2021-01-28 13:20:00,откуда взялся долг кредитной карте карту никог...,3,ооо,Where did the credit card debt come from if I ...,credit card debt come never issued card bank l...,0.910927
9,fk_otkritie,"Уважаемый Бинбанк, если у вас имеются какие-ли...",1.0,2018-08-31 16:48:00,уважаемый бинбанк вас имеются какиелибо вопрос...,1,кк,"Dear Binbank, if you have any questions, pleas...",dear binbank questions please contact phone nu...,0.936102


In [101]:
miss.to_csv('train_en_w_miss.csv', index=False)

In [107]:
add = miss[((miss.grades == 1)&(miss.prob>0.5))|((miss.grades != 1)&(miss.prob>0.7))]

In [111]:
train_w_add_en = pd.concat([train_notna, add], axis=0)

In [113]:
train_w_add_en.drop(columns=['prob'], inplace=True)

In [116]:
train_w_add_en.to_csv('train_en_w_miss.csv', index=False)

# Попробуем посмотреть на окончания цитат

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words,en,lemmas
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,2017-02-16 16:10:00,лет являюсь клиентом этого банка но последний ...,7,отвратительное отношение клиентам ой ужас,I have been a client of this bank for many yea...,client bank many years last visit left negativ...
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,2016-12-13 01:05:00,ростовнадону ул ленина часов в данном офисе не...,3,г в ао,"Rostov-on-Don, st. Lenina, 48. Were on Decembe...",rostov lenina december single competent employ...
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2.0,2020-04-08 06:38:00,имею потребительский кредит взятый в связьбанк...,4,в в псб,I have a consumer loan taken from Svyaz-Bank a...,consumer loan taken svyaz bank transferred pro...
7,homecreditbank,Хочу выразить глубокую благодарность всем сотр...,5.0,2015-04-19 20:11:00,хочу выразить глубокую благодарность всем сотр...,0,NaN,I want to express my deep gratitude to all emp...,want express deep gratitude employees high lev...
8,vtb,"6 марта 2019г. В 10:33 при проходе в метро ""Ку...",2.0,2019-03-28 18:36:00,марта в проходе в метро курская социальной кар...,3,в втб,"March 6, 2019 At 10:33 when I was going to the...",march going metro station kurskaya social card...


# Эксперимент

In [3]:
path1= 'Эксперимент 2/prep_train.csv'
path2 = 'Эксперимент 2/train_prep_en.csv'

In [4]:
df1_tr = pd.read_csv(path1)
df2_tr = pd.read_csv(path2)

In [5]:
df1_tr.head()

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,2017-02-16 16:10:00,лет являюсь клиентом этого банка но последний ...,7,отвратительное отношение клиентам ой ужас
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,2016-12-13 01:05:00,ростовнадону ул ленина часов в данном офисе не...,3,г в ао
2,alfabank,Здравствуйте!Ранее уже оставлял отзыв о вашем ...,NaN,2019-06-28 13:54:00,здравствуйте восклицание ранее оставлял отзыв ...,4,в но а фио
3,vtb,Обращаюсь к Вам с жалобой на незаконное списан...,NaN,2020-07-15 14:54:00,обращаюсь вам жалобой на незаконное списание д...,30,пао втб уфк пао втб пао втб n фз ...
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2.0,2020-04-08 06:38:00,имею потребительский кредит взятый в связьбанк...,4,в в псб


In [6]:
df2_tr.head()

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words,en,lemmas
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,2017-02-16 16:10:00,лет являюсь клиентом этого банка но последний ...,7,отвратительное отношение клиентам ой ужас,I have been a client of this bank for many yea...,client bank many years last visit left negativ...
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,2016-12-13 01:05:00,ростовнадону ул ленина часов в данном офисе не...,3,г в ао,"Rostov-on-Don, st. Lenina, 48. Were on Decembe...",rostov lenina december single competent employ...
2,alfabank,Здравствуйте!Ранее уже оставлял отзыв о вашем ...,NaN,2019-06-28 13:54:00,здравствуйте восклицание ранее оставлял отзыв ...,4,в но а фио,Hello! I have already left a review about your...,hello already left review bank incompetence em...
3,vtb,Обращаюсь к Вам с жалобой на незаконное списан...,NaN,2020-07-15 14:54:00,обращаюсь вам жалобой на незаконное списание д...,30,пао втб уфк пао втб пао втб n фз ...,I am writing to you with a complaint about the...,writing complaint illegal debiting funds child...
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2.0,2020-04-08 06:38:00,имею потребительский кредит взятый в связьбанк...,4,в в псб,I have a consumer loan taken from Svyaz-Bank a...,consumer loan taken svyaz bank transferred pro...


In [8]:
path11= 'Эксперимент 2/prep_test.csv'
path22 = 'Эксперимент 2/test_prep_en.csv'

In [9]:
df1_test = pd.read_csv(path11)
df2_test = pd.read_csv(path22)

In [10]:
df1_test.head()

,bank,feeds,date,prep_feeds,cups_num,cups_words
0,sberbank,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,01.07.2020 10:53,оформляем ипотеку в сбербанке подгружены необх...,0,NaN
1,alfabank,Краткое содержание: не рекомендую брать кредит...,20.06.2019 13:19,краткое содержание не рекомендую брать кредит ...,3,в в в
2,v-express-bank,"Добрый день, уважаемые сотрудники службы контр...",20.02.2016 11:46,добрый день уважаемые сотрудники службы контро...,3,м цб
3,homecreditbank,"Обращался за получением карты ""Зеленая польза""...",06.05.2019 15:48,обращался получением карты зеленая польза сотр...,0,NaN
4,vtb,20.05.2016 обратилась в отделение банка на про...,23.05.2016 15:41,обратилась в отделение банка на проспекте лени...,9,втб втб втб бесплатно колоссальный боль...


In [11]:
df2_test.head()

,bank,feeds,date,prep_feeds,cups_num,cups_words,en,lemmas
0,sberbank,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,01.07.2020 10:53,оформляем ипотеку в сбербанке подгружены необх...,0,NaN,We arrange a mortgage in Sberbank. On 06/22/20...,arrange mortgage sberbank necessary documents ...
1,alfabank,Краткое содержание: не рекомендую брать кредит...,20.06.2019 13:19,краткое содержание не рекомендую брать кредит ...,3,в в в,Short content: I do not recommend taking a loa...,short content recommend taking loan bank clien...
2,v-express-bank,"Добрый день, уважаемые сотрудники службы контр...",20.02.2016 11:46,добрый день уважаемые сотрудники службы контро...,3,м цб,"Good afternoon, dear employees of the quality ...",good afternoon dear employees quality control ...
3,homecreditbank,"Обращался за получением карты ""Зеленая польза""...",06.05.2019 15:48,обращался получением карты зеленая польза сотр...,0,NaN,"I applied for a ""Green Benefit"" card, the empl...",applied green benefit card employees said inst...
4,vtb,20.05.2016 обратилась в отделение банка на про...,23.05.2016 15:41,обратилась в отделение банка на проспекте лени...,9,втб втб втб бесплатно колоссальный боль...,"On May 20, 2016, she applied to the bank branc...",applied bank branch lenina prospekt mortgage c...


# Приготовим данные и обучим модели

In [12]:
df1_tr.date = pd.to_datetime(df1_tr.date)
df1_tr['y'] = df1_tr.date.dt.year
df1_tr['m'] = df1_tr.date.dt.month

In [15]:
df2_tr.date = pd.to_datetime(df2_tr.date)
df2_tr['y'] = df2_tr.date.dt.year
df2_tr['m'] = df2_tr.date.dt.month

In [17]:
df1_test.date = pd.to_datetime(df1_test.date)
df1_test['y'] = df1_test.date.dt.year
df1_test['m'] = df1_test.date.dt.month

In [16]:
df2_test.date = pd.to_datetime(df2_test.date)
df2_test['y'] = df2_test.date.dt.year
df2_test['m'] = df2_test.date.dt.month

In [18]:
banks = df1_tr.bank.value_counts().index.tolist()

In [19]:
years = df1_tr.y.value_counts().index.tolist()

In [22]:
def bank_rating(banks, years):
    means = {}
    for b in banks:
        means[b] = {}
        for y in years:
            means[b][y] = df1_tr[(df1_tr.bank == b) & (df1_tr.y == y)].grades.mean()
    return means

In [23]:
# рейтинг банков по годам
b_rating = bank_rating(banks, years)

In [24]:
mean_rating = df1_tr.grades.mean()

In [25]:
mean_rating

2.395096744113762

In [26]:
rating = []
for i, item in tqdm(enumerate(df1_tr.iterrows())):
    tmp = b_rating.get(item[1][0], {})
    res = tmp.get(item[1][-2], mean_rating)
    rating.append(res)
    #if i == 6: break

75000it [00:03, 23108.30it/s]


In [27]:
df1_tr['rating'] = rating
df2_tr['rating'] = rating

In [28]:
# Добавим длину коментов
df1_tr['len'] = df1_tr.feeds.apply(lambda x: len(x))
df2_tr['len'] = df2_tr.feeds.apply(lambda x: len(x))

In [32]:
df1_tr.head(1)

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words,y,m,rating,len
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,2017-02-16 16:10:00,лет являюсь клиентом этого банка но последний ...,7,отвратительное отношение клиентам ой ужас,2017,2,2.0,1286


# Test

In [34]:
rating = []
for i, item in tqdm(enumerate(df1_test.iterrows())):
    tmp = b_rating.get(item[1][0], {})
    res = tmp.get(item[1][-2], mean_rating)
    rating.append(res)

17220it [00:01, 15003.16it/s]


In [35]:
df1_test['rating'] = rating
df2_test['rating'] = rating

In [36]:
# Добавим длину коментов
df1_test['len'] = df1_test.feeds.apply(lambda x: len(x))
df2_test['len'] = df2_test.feeds.apply(lambda x: len(x))

In [37]:
df1_test.head(2)

,bank,feeds,date,prep_feeds,cups_num,cups_words,y,m,rating,len
0,sberbank,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,2020-01-07 10:53:00,оформляем ипотеку в сбербанке подгружены необх...,0,NaN,2020,1,1.938999,331
1,alfabank,Краткое содержание: не рекомендую брать кредит...,2019-06-20 13:19:00,краткое содержание не рекомендую брать кредит ...,3,в в в,2019,6,2.174785,1048


# Сделаем лемматизацию для 1 df

In [39]:
df1_tr.head()

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words,y,m,rating,len
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,2017-02-16 16:10:00,лет являюсь клиентом этого банка но последний ...,7,отвратительное отношение клиентам ой ужас,2017,2,2.000000,1286
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,2016-12-13 01:05:00,ростовнадону ул ленина часов в данном офисе не...,3,г в ао,2016,12,1.940767,1000
2,alfabank,Здравствуйте!Ранее уже оставлял отзыв о вашем ...,NaN,2019-06-28 13:54:00,здравствуйте восклицание ранее оставлял отзыв ...,4,в но а фио,2019,6,2.174785,1155
3,vtb,Обращаюсь к Вам с жалобой на незаконное списан...,NaN,2020-07-15 14:54:00,обращаюсь вам жалобой на незаконное списание д...,30,пао втб уфк пао втб пао втб n фз ...,2020,7,2.038381,2117
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2.0,2020-04-08 06:38:00,имею потребительский кредит взятый в связьбанк...,4,в в псб,2020,4,2.088083,1614


In [41]:
m = MorphAnalyzer()
regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('russian') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [42]:
lemmas = []
for i, text in tqdm(enumerate(df1_tr['prep_feeds'].tolist())):
    lemmas.append(clean_text(text))

75000it [21:30, 58.11it/s]


In [43]:
df1_tr['lemmas'] = lemmas

In [44]:
lemmas = []
for i, text in tqdm(enumerate(df1_test['prep_feeds'].tolist())):
    lemmas.append(clean_text(text))

17220it [04:59, 57.52it/s]


In [ ]:
df1_test['lemmas'] = lemmas

# Сделаем предикт на английских текстах и на русских

# TF-IDF

In [45]:
vect_words = TfidfVectorizer(max_features=250000, analyzer='word', ngram_range=(1, 3))
vect_chars = TfidfVectorizer(max_features=12000, analyzer='char', ngram_range=(1, 3))

In [46]:
all_words_train = vect_words.fit_transform(df2_tr[df2_tr.grades.notna()].lemmas)
all_chars_train = vect_chars.fit_transform(df2_tr[df2_tr.grades.notna()].lemmas)

In [47]:
all_words_train.shape

(51476, 250000)

In [48]:
all_chars_train.shape

(51476, 11291)

In [49]:
all_words_miss = vect_words.transform(df2_tr[df2_tr.grades.isna()].lemmas)
all_chars_miss = vect_chars.transform(df2_tr[df2_tr.grades.isna()].lemmas)

In [50]:
all_words_test = vect_words.transform(df2_test.lemmas)
all_chars_test = vect_chars.transform(df2_test.lemmas)

In [51]:
# Соберем матрицу признаков TF-IDF для теста и трейна
train_feats = sparse.hstack([all_words_train, all_chars_train])

In [52]:
miss_feats = sparse.hstack([all_words_miss, all_chars_miss])

In [53]:
test_feats = sparse.hstack([all_words_test, all_chars_test])

# Загрузим модель и сделаем предикт

In [54]:
# Загрузим модельку
# tf-idf word250k+char12k
with open('model_7_en.pickle', 'rb') as f:
    model_7_en_LR_CV5 = pickle.load(f)

In [55]:
# Сделаем прогноз на пропусках
pred_miss_en = model_7_en_LR_CV5.predict(miss_feats)
pred_miss_prob_en = model_7_en_LR_CV5.predict_proba(miss_feats)

In [56]:
pred_train_en = model_7_en_LR_CV5.predict(train_feats)
pred_train_prob_en = model_7_en_LR_CV5.predict_proba(train_feats)

In [57]:
pred_test_en = model_7_en_LR_CV5.predict(test_feats)
pred_test_en_prob = model_7_en_LR_CV5.predict_proba(test_feats)

In [68]:
df2_tr['predict'] = 1
df2_tr['predict_prob'] = 0.2

In [80]:
#train
df2_tr.loc[df2_tr[df2_tr.grades.notna()].index, ['predict']] = pred_train_en
df2_tr.loc[df2_tr[df2_tr.grades.notna()].index, ['predict_prob']] = pred_train_prob_en.max(axis=1)

In [84]:
#miss
df2_tr.loc[df2_tr[df2_tr.grades.isna()].index, ['predict']] = pred_miss_en
df2_tr.loc[df2_tr[df2_tr.grades.isna()].index, ['predict_prob']] = pred_miss_prob_en.max(axis=1)
df2_tr.loc[df2_tr[df2_tr.grades.isna()].index, ['grades']] = pred_miss_en

In [86]:
#test
df2_test['predict'] = pred_test_en
df2_test['predict_prob'] = pred_test_en_prob.max(axis=1)

In [87]:
df2_tr.head(2)

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words,en,lemmas,y,m,rating,len,predict,predict_prob
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,2017-02-16 16:10:00,лет являюсь клиентом этого банка но последний ...,7,отвратительное отношение клиентам ой ужас,I have been a client of this bank for many yea...,client bank many years last visit left negativ...,2017,2,2.000000,1286,1,0.932908
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,2016-12-13 01:05:00,ростовнадону ул ленина часов в данном офисе не...,3,г в ао,"Rostov-on-Don, st. Lenina, 48. Were on Decembe...",rostov lenina december single competent employ...,2016,12,1.940767,1000,2,0.570063


# Теперь русскую модель

In [90]:
stem_rt = pd.read_csv("Эксперимент 2/stem_prep_train.csv")
stem_test = pd.read_csv("Эксперимент 2/stem_prep_test.csv")

In [93]:
stem_rt.head()

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words,stem_text
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,2017-02-16 16:10:00,лет являюсь клиентом этого банка но последний ...,7,отвратительное отношение клиентам ой ужас,лет явля клиент эт банк но последн виз прост о...
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,2016-12-13 01:05:00,ростовнадону ул ленина часов в данном офисе не...,3,г в ао,ростовнадон ул ленин час в дан офис не оказа н...
2,alfabank,Здравствуйте!Ранее уже оставлял отзыв о вашем ...,NaN,2019-06-28 13:54:00,здравствуйте восклицание ранее оставлял отзыв ...,4,в но а фио,здравств восклицан ран оставля отз ваш банк не...
3,vtb,Обращаюсь к Вам с жалобой на незаконное списан...,NaN,2020-07-15 14:54:00,обращаюсь вам жалобой на незаконное списание д...,30,пао втб уфк пао втб пао втб n фз ...,обраща вам жалоб на незакон списан денежн сред...
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2.0,2020-04-08 06:38:00,имею потребительский кредит взятый в связьбанк...,4,в в псб,им потребительск кред взят в связьбанк перешед...


In [94]:
vect_words = TfidfVectorizer(max_features=50000, analyzer='word', ngram_range=(1, 3))
vect_chars = TfidfVectorizer(max_features=25000, analyzer='char', ngram_range=(1, 3))

In [105]:
all_words_train = vect_words.fit_transform(stem_rt.stem_text)
all_chars_train = vect_chars.fit_transform(stem_rt.stem_text)

In [97]:
all_words_train.shape

(51476, 50000)

In [98]:
#all_words_miss = vect_words.transform(stem_rt[stem_rt.grades.isna()].stem_text)
#all_chars_miss = vect_chars.transform(stem_rt[stem_rt.grades.isna()].stem_text)

In [112]:
all_words_test = vect_words.transform(stem_test.stem_text)
all_chars_test = vect_chars.transform(stem_test.stem_text)

In [100]:
all_words_test.shape

(17220, 50000)

In [113]:
# Соберем матрицу признаков TF-IDF для теста и трейна
train_feats_ru = sparse.hstack([all_words_train, all_chars_train])
#train_miss_ru = sparse.hstack([all_words_miss, all_chars_miss])
test_feats_ru = sparse.hstack([all_words_test, all_chars_test])

In [117]:
with open('model_tfidf_BigData_3.pickle', 'rb') as f:
    model_3_LR_CV3 = pickle.load(f)

In [118]:
# Сделаем прогноз на пропусках
#pred_miss_ru = model_3_LR_CV3.predict(train_miss_ru)
#pred_miss_prob_ru = model_3_LR_CV3.predict_proba(train_miss_ru)

In [119]:
pred_train_ru = model_3_LR_CV3.predict(train_feats_ru)
pred_train_prob_ru = model_3_LR_CV3.predict_proba(train_feats_ru)

In [120]:
pred_test_ru = model_3_LR_CV3.predict(test_feats_ru)
pred_test_ru_prob = model_3_LR_CV3.predict_proba(test_feats_ru)

In [ ]:
#df1_tr['predict'] = 1
#df1_tr['predict_prob'] = 0.2

In [121]:
#train
df1_tr['predict'] = pred_train_ru
df1_tr['predict_prob'] = pred_train_prob_ru.max(axis=1)

In [122]:
#miss
#df2_tr.loc[df2_tr[df2_tr.grades.isna()].index, ['predict']] = pred_miss_en
#df2_tr.loc[df2_tr[df2_tr.grades.isna()].index, ['predict_prob']] = pred_miss_prob_en.max(axis=1)
df1_tr.loc[df1_tr[df1_tr.grades.isna()].index, ['grades']] = pred_miss_en

In [123]:
#test
df1_test['predict'] = pred_test_ru
df1_test['predict_prob'] = pred_test_ru_prob.max(axis=1)

# Посмотрим что мы имеем

In [126]:
#del(stem_rt)
del(stem_test)

In [127]:
df1_tr.head(2)

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words,y,m,rating,len,lemmas,predict,predict_prob
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,2017-02-16 16:10:00,лет являюсь клиентом этого банка но последний ...,7,отвратительное отношение клиентам ой ужас,2017,2,2.000000,1286,являться клиент банк последний визит просто ос...,1,0.942412
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,2016-12-13 01:05:00,ростовнадону ул ленина часов в данном офисе не...,3,г в ао,2016,12,1.940767,1000,ростовнадон ленин данный офис оказаться компет...,1,0.881680


In [128]:
df2_tr.head(2)

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words,en,lemmas,y,m,rating,len,predict,predict_prob
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,2017-02-16 16:10:00,лет являюсь клиентом этого банка но последний ...,7,отвратительное отношение клиентам ой ужас,I have been a client of this bank for many yea...,client bank many years last visit left negativ...,2017,2,2.000000,1286,1,0.932908
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,2016-12-13 01:05:00,ростовнадону ул ленина часов в данном офисе не...,3,г в ао,"Rostov-on-Don, st. Lenina, 48. Were on Decembe...",rostov lenina december single competent employ...,2016,12,1.940767,1000,2,0.570063


In [129]:
df1_test.head(2)

,bank,feeds,date,prep_feeds,cups_num,cups_words,y,m,rating,len,predict,predict_prob
0,sberbank,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,2020-01-07 10:53:00,оформляем ипотеку в сбербанке подгружены необх...,0,NaN,2020,1,1.938999,331,1,0.959757
1,alfabank,Краткое содержание: не рекомендую брать кредит...,2019-06-20 13:19:00,краткое содержание не рекомендую брать кредит ...,3,в в в,2019,6,2.174785,1048,1,0.887045


In [130]:
df2_test.head(2)

,bank,feeds,date,prep_feeds,cups_num,cups_words,en,lemmas,y,m,rating,len,predict,predict_prob
0,sberbank,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,2020-01-07 10:53:00,оформляем ипотеку в сбербанке подгружены необх...,0,NaN,We arrange a mortgage in Sberbank. On 06/22/20...,arrange mortgage sberbank necessary documents ...,2020,1,1.938999,331,1.0,0.651007
1,alfabank,Краткое содержание: не рекомендую брать кредит...,2019-06-20 13:19:00,краткое содержание не рекомендую брать кредит ...,3,в в в,Short content: I do not recommend taking a loa...,short content recommend taking loan bank clien...,2019,6,2.174785,1048,1.0,0.883919


# Стакнем все данные в кучу

In [149]:
TRAIN = df1_tr.loc[:, ['bank', 'y', 'm', 'cups_num', 'rating', 'len', 'predict', 'predict_prob']]

In [150]:
TRAIN['predict_en'] = df2_tr.predict
TRAIN['predict_prob_en'] = df2_tr.predict_prob
TRAIN['grades'] = df2_tr.grades

In [151]:
TRAIN.head()

,bank,y,m,cups_num,rating,len,predict,predict_prob,predict_en,predict_prob_en,grades
0,ubrr,2017,2,7,2.000000,1286,1,0.942412,1,0.932908,1.0
1,fk_otkritie,2016,12,3,1.940767,1000,1,0.881680,2,0.570063,2.0
2,alfabank,2019,6,4,2.174785,1155,1,0.935093,1,0.841549,1.0
3,vtb,2020,7,30,2.038381,2117,1,0.882855,1,0.976208,1.0
4,promsvyazbank,2020,4,4,2.088083,1614,1,0.798029,1,0.494922,2.0


In [139]:
TEST = df1_test.loc[:, ['bank', 'y', 'm', 'cups_num', 'rating', 'len', 'predict', 'predict_prob']]

In [140]:
TEST['predict_en'] = df2_test.predict
TEST['predict_prob_en'] = df2_test.predict_prob

In [141]:
TEST.head()

,bank,y,m,cups_num,rating,len,predict,predict_prob,predict_en,predict_prob_en
0,sberbank,2020,1,0,1.938999,331,1,0.959757,1.0,0.651007
1,alfabank,2019,6,3,2.174785,1048,1,0.887045,1.0,0.883919
2,v-express-bank,2016,2,3,1.367347,731,1,0.807355,1.0,0.816662
3,homecreditbank,2019,6,0,2.877598,415,1,0.909200,1.0,0.935203
4,vtb,2016,5,9,2.262681,2510,1,0.790558,1.0,0.816021


In [142]:
TRAIN.shape

(75000, 11)

In [143]:
TEST.shape

(17220, 10)

# OHE

In [144]:
ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [0, 1, 2])],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough')

In [153]:
ohe_train = ct.fit_transform(TRAIN.loc[:, ['bank', 'y', 'm']])

In [154]:
ohe_train

<75000x98 sparse matrix of type '<class 'numpy.float64'>'
	with 225000 stored elements in Compressed Sparse Row format>

In [155]:
df_ohe = pd.DataFrame(ohe_train.toarray(), index=TRAIN.index)

In [160]:
TRAIN_ohe = pd.concat([df_ohe, TRAIN.iloc[:, 3:]], axis=1)

In [161]:
ohe_test = ct.transform(TEST.loc[:, ['bank', 'y', 'm']])

In [162]:
df_ohe_t = pd.DataFrame(ohe_test.toarray(), index=TEST.index)

In [163]:
TEST_ohe = pd.concat([df_ohe_t, TEST.iloc[:, 3:]], axis=1)

In [164]:
# ИТОГО ПОЛУЧИЛИ
TRAIN_ohe.head(2)

,0,1,2,3,4,5,6,7,8,9,...,96,97,cups_num,rating,len,predict,predict_prob,predict_en,predict_prob_en,grades
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7,2.000000,1286,1,0.942412,1,0.932908,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,3,1.940767,1000,1,0.881680,2,0.570063,2.0


In [166]:
TEST_ohe.head(2)

,0,1,2,3,4,5,6,7,8,9,...,95,96,97,cups_num,rating,len,predict,predict_prob,predict_en,predict_prob_en
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,1.938999,331,1,0.959757,1.0,0.651007
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3,2.174785,1048,1,0.887045,1.0,0.883919


In [171]:
TRAIN_ohe.to_csv('TRAIN_ohe.csv', index=False)

In [ ]:
TEST_ohe.to_csv('TEST_ohe.csv', index=False)

# Попробуем AutoML

In [ ]:
#!pip install h2o

In [169]:
from h2o.automl import H2OAutoML
import h2o

In [174]:
# установим максимальный размер используемой оперативной памяти
h2o.init(max_mem_size='8G')

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,46 secs
H2O_cluster_timezone:,Europe/Moscow
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_kulak_71hfe8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [175]:
train_aml = h2o.H2OFrame(TRAIN_ohe)
test_aml = h2o.H2OFrame(TEST_ohe)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [176]:
x = train_aml.columns[:-1]
y = 'grades'

In [177]:
aml = H2OAutoML(max_models=15,  		# Количество различных моделей для обучения
                seed=42, 
                max_runtime_secs = 31000)   # Максимальное время обучения одной модели
aml.train(x=x, y=y, training_frame=train_aml)

# Ждем, когда заполнится строка обучения AutoML

AutoML progress: |█
17:42:51.207: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.1379516,0.0012550,0.1391261,0.1377923,0.1365954,0.1368979,0.1393464
mean_residual_deviance,0.1645065,0.0052771,0.1645858,0.1603754,0.1588380,0.1721003,0.166633
mse,0.1645065,0.0052771,0.1645858,0.1603754,0.1588380,0.1721003,0.166633
null_deviance,40831.277,327.72556,40678.246,40797.02,40718.117,41398.086,40564.918
r2,0.9395657,0.0014594,0.9393699,0.9409139,0.9412000,0.937938,0.9384067
residual_deviance,2467.3674,70.15982,2466.3179,2410.4417,2393.847,2567.7356,2498.495
rmse,0.4055525,0.0064895,0.4056917,0.4004689,0.3985448,0.4148497,0.4082070
rmsle,0.1248019,0.0011463,0.1254887,0.1245354,0.1229466,0.1258673,0.1251713


In [178]:
preds_train = aml.predict(train_aml).as_data_frame().values
preds_train.astype(int)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


array([[1],
       [2],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [179]:
f1_score(preds_train.astype(int), TRAIN_ohe.grades, average='micro')

0.6505466666666667

In [180]:
preds = aml.predict(test_aml).as_data_frame().values

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [181]:
preds = preds.astype(int)

In [185]:
preds.flatten()

array([1, 1, 1, ..., 1, 1, 1])

In [186]:
sol = pd.DataFrame({'inds': df1_test.index,
                    'grades': preds.flatten()})

In [188]:
sol.grades.value_counts()

1    9955
4    3138
5    1821
0    1664
2     599
3      43
Name: grades, dtype: int64

In [189]:
sol.to_csv('solution_4.csv', index=False)

## Скор на Kaggle 0.558

# CatBoost

In [227]:
TRAIN = df1_tr.loc[:, ['feeds', 'prep_feeds', 'bank', 'y', 'm', 'cups_num', 'rating', 'len', 'predict', 'predict_prob']]

In [228]:
TRAIN['feeds_en'] = df2_tr.en
TRAIN['lemmas_en'] = df2_tr.lemmas
TRAIN['predict_en'] = df2_tr.predict
TRAIN['predict_prob_en'] = df2_tr.predict_prob
TRAIN['grades'] = df2_tr.grades

In [229]:
TEST = df1_test.loc[:, ['feeds', 'prep_feeds', 'bank', 'y', 'm', 'cups_num', 'rating', 'len', 'predict', 'predict_prob']]

In [230]:
TEST['feeds_en'] = df2_test.en
TEST['lemmas_en'] = df2_test.lemmas
TEST['predict_en'] = df2_test.predict
TEST['predict_prob_en'] = df2_test.predict_prob

In [231]:
TRAIN.head(2)

,feeds,prep_feeds,bank,y,m,cups_num,rating,len,predict,predict_prob,feeds_en,lemmas_en,predict_en,predict_prob_en,grades
0,"Много лет являюсь клиентом этого банка, но пос...",лет являюсь клиентом этого банка но последний ...,ubrr,2017,2,7,2.000000,1286,1,0.942412,I have been a client of this bank for many yea...,client bank many years last visit left negativ...,1,0.932908,1.0
1,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",ростовнадону ул ленина часов в данном офисе не...,fk_otkritie,2016,12,3,1.940767,1000,1,0.881680,"Rostov-on-Don, st. Lenina, 48. Were on Decembe...",rostov lenina december single competent employ...,2,0.570063,2.0


In [232]:
TEST.head(2)

,feeds,prep_feeds,bank,y,m,cups_num,rating,len,predict,predict_prob,feeds_en,lemmas_en,predict_en,predict_prob_en
0,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,оформляем ипотеку в сбербанке подгружены необх...,sberbank,2020,1,0,1.938999,331,1,0.959757,We arrange a mortgage in Sberbank. On 06/22/20...,arrange mortgage sberbank necessary documents ...,1.0,0.651007
1,Краткое содержание: не рекомендую брать кредит...,краткое содержание не рекомендую брать кредит ...,alfabank,2019,6,3,2.174785,1048,1,0.887045,Short content: I do not recommend taking a loa...,short content recommend taking loan bank clien...,1.0,0.883919


In [236]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [247]:
model_cbc = CatBoostClassifier(iterations=25000,
                           eval_metric='MultiClass',
                           learning_rate=0.009,
                           custom_loss=['F1', 'TotalF1'],
                           use_best_model=True,
                           verbose=False)

In [240]:
X_train, X_test, y_train, y_test = train_test_split(TRAIN.iloc[:, :-1], TRAIN.iloc[:, -1], train_size=0.9, random_state=42)

In [251]:
model_cbc.fit(X_train, y_train,
          verbose=False,
          text_features=['feeds', 'prep_feeds', 'feeds_en', 'lemmas_en'],
          cat_features=['bank', 'y', 'm', 'predict', 'predict_en'],
          eval_set= (X_test, y_test),
          plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [254]:
TEST.predict_en = TEST.predict_en.astype(int)

In [256]:
predict_sub = model_cbc.predict(TEST)

In [263]:
predict_sub = predict_sub.astype(int).flatten()

In [264]:
sol = pd.DataFrame({'inds': df1_test.index,
                    'grades': predict_sub})

In [265]:
sol

,inds,grades
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
17215,17215,1
17216,17216,1
17217,17217,1
17218,17218,1


In [266]:
sol.grades.value_counts()

1    11669
5     4952
2      457
3       93
4       49
Name: grades, dtype: int64

In [267]:
sol.to_csv('solution_5.csv', index=False)

In [268]:
# Скор 0.78